# CHECK Spectraction Statistics

- author : Sylvie Dagoret-Campagne
- creation date : 2025-10-06
- last update : 2025-10-06

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

plt.rcParams["figure.figsize"] = (16,8)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'
plt.rcParams["legend.fontsize"] = "small"
pd.set_option('display.max_rows', 100)

In [ ]:
#import ipywidgets as widgets
#%matplotlib widget

In [ ]:
# CHECK THE CONFIG HERE !!!!!!
from BUTLER00_parameters import *

In [ ]:
DumpConfig()

In [ ]:
extractedfilesdict.keys()

In [ ]:
legendtag

In [ ]:
#list_of_Keys = extractedfilesdict.keys()

In [ ]:
#d_table = {}
#for key in list_of_Keys:
#    path_file  = extractedfilesdict[key]
#    data = np.load(path_file,allow_pickle=True)
#    n = len(data)
#    print(key,legendtag[key],n)   

## Configuration

In [ ]:
selected_run = version_run

In [ ]:
inputfullfilename = extractedfilesdict[selected_run]
print(inputfullfilename )

In [ ]:
the_collection = butlerusercollectiondict[selected_run] 

In [ ]:
the_tag = legendtag[selected_run] 

In [ ]:
the_suptitle = the_tag + f" ({the_collection})"

In [ ]:
the_collection_name = the_collection.replace("/","_")

## Read file

In [ ]:
data = np.load(inputfullfilename,allow_pickle=True)

In [ ]:
df = pd.DataFrame(data)

In [ ]:
print(list(df.columns))

## Computations

In [ ]:
df["nightObs"] = df.apply(lambda x: x['id']//100_000, axis=1)

In [ ]:
df["seq_num"] = df["id"] - df["nightObs"]*100_000

In [ ]:
df["date"] = pd.to_datetime(df["DATE-OBS"])

In [ ]:
df.FILTER.unique()

In [ ]:
plt.figure(figsize=(20,8))

# conversion en datetime
#df["date"] = pd.to_datetime(df["DATE-OBS"].astype(str), format="%Y%m%d")


sns.scatterplot(
    data=df, 
    x="date",       # abscisse en datetime
    y="seq_num",    # ou ra, dec, etc.
    hue="FILTER", 
    palette="tab10"
)

plt.title(f"Auxtel Holo observations wrt date and filter type")
plt.xlabel("Date of observation")
plt.ylabel("Seq Num")
plt.xticks(rotation=45)  # lisibilité des dates
#plt.legend(loc="upper left",ncol=8)
plt.legend(bbox_to_anchor=(1.01, 1.05),ncols=1)
plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np

# on décale légèrement la position en y en fonction de seq_num
df["FILTER_seq"] = df["FILTER"].astype(str) + "_" + df["seq_num"].astype(str)

plt.figure(figsize=(20,8))
sns.stripplot(
    data=df,
    x="date",
    y="FILTER",
    hue="FILTER",
    palette="Set1",
    size=4,         # taille des points
    jitter=True,    # évite que les points se chevauchent
    alpha=1.0
)

plt.title(f"Auxtel Holo observations wrt date and filter type",fontsize=15)
plt.xlabel("Time")
plt.ylabel("Filter")
plt.xticks(rotation=45)
#plt.legend(loc="upper left", ncol=6)
plt.grid()
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(20,16))
df["TARGET_seq"] = df["TARGET"].astype(str) + "_" + df["seq_num"].astype(str)
sns.stripplot(
    data=df,
    x="date",
    y="TARGET",
    hue="TARGET",
    palette="Set2",
    size=10,         # taille des points
    jitter=True,    # évite que les points se chevauchent
    alpha=1.0,
    edgecolor="black",
    linewidth=0.1
)
plt.title(f"Auxtel Holo observations wrt date and target",fontsize=15)
plt.xlabel("Time")
plt.ylabel("Target")
plt.xticks(rotation=45)
plt.tight_layout()
plt.grid()
plt.show()